# Hadamard Pooling — Analysis Vector Checking Notebook
This streamlined notebook does exactly the following:
1) Input **n**
2) Print *S* matrix
3) Print type-2 reduced *S* matrix
4) Print pool assignments (A, F, G, …)
5) Show an example analysis vector
6) Accept a **results vector** and identify **one or more positive samples** (sum of columns)

First run all cells, then in widget in final cell:

Input desired matrix size then press 'build S matrix'

Next:

Input Analysis Vector as a comma or space separated list e.g. 1,1,0,0 or 1 1 0 0 then press 'check' - this outputs the positive sample or samples.






In [119]:
import numpy as np
import ipywidgets as wgt
from IPython.display import display, Markdown
# import all python 3 add-ons etc that will be needed later on
import numpy.linalg as La
import time
from itertools import combinations
import ipywidgets.widgets  as wgt
from ipywidgets import interact, interactive, fixed, interact_manual,VBox,HBox,Layout,Output
from IPython.display import display

In [120]:
#Quadratic Residues Construction

def valid_seq_length(n):              # check Hadamard sequence length for Quadratic Residue method only
    maxi = 230
    Hseq = np.zeros(maxi,dtype=int)
    for i in range(maxi):                   # produce Hadamard sequence numbers
        for m in range(0,maxi):
            if isprime(i) and i == 4*m + 3:
                Hseq[i]=i
            pass
    if n in Hseq[0:maxi]:
        is_ok=True
    else:
        is_ok=False
    return is_ok

# check if integer n is a prime, range starts with 2 and only needs to go up the squareroot of n
def isprime(n):
    for x in range(2, int(n**0.5)+1):
        if n % x == 0:
            return False
    return True
#------------
# quadratic residue method, this generates more S matrices than the other methods.
def quadratic_hadamard(n):

    init_list = np.zeros(n,dtype=int)

    for i in range(n):                     # make list n/2+1 values = 1 rest zeros so in same ratio as hadamard
        if i <= n//2 : init_list[i] = 1    # check integer division
        pass
    alist = np.zeros(n,dtype=int)
    for i in range(0,(n-1)//2):            # integer division
        alist[(i+1)*(i+1) % n] = 1         # alist = hadamard Srow need only go to half range of n as indices are symmetric
    alist[0] = 1
    Srow = list(alist)

    S = np.zeros((n,n),dtype = int)
    for i in range(n):
        for j in range(n):
            S[i,j]= Srow[n-1-j]
        Srow = np.roll(Srow, 1)             # rotate by 1 element at a time
        pass

    return S                               # returns S matrix
#------------
#print('Hadamard S matrices by Quadratic residue method')
#for i in range(1,200):
#    if valid_seq_length(i):
#        S = quadratic_hadamard(i)
#        print(i)
#        if i  <= 32 :
#            print('\n'.join( [''.join(['{:2}'.format(item) for item in row] ) for row in S] ) )
#        else:
#            print(''.join(['{:2}'.format(item) for item in S[0]]))
#            xs=''.join( str(S[0][i]) for i in range(len(S[0])) )
#            print(hex(int(xs,2)))
#    pass

In [121]:
#-------------
# shift register method S matrix size  2^k-1 , k = 2, 3, 4  etc
def shift_hadamard(num):
    num = int(num) + 1
    def S_lineA(n,m):
        SL     = np.zeros(n,dtype=int)
        SL[0] = 1                               # set x^n = 1
        for j in range(2**n-1):

            tmp = (SL[m] + SL[0]) % 2           # mod 2
            SL = np.roll(SL,-1)                 # shift array elements
            SL[n-1] = tmp                       # last one as x^n
            Srow[j] = SL[0]
        pass

    def S_lineB(n,ma,mb,mc):
        SL    = np.zeros(n,dtype=int)
        SL[0] = 1                               # set x^n = 1
        for j in range(2**n-1):
            tmp1 = (SL[ma] + SL[0]) % 2         # mod 2
            tmp2 = (SL[mb] + tmp1 ) % 2
            tmp  = (SL[mc] + tmp2 ) % 2
            SL = np.roll(SL,-1)
            SL[n-1] = tmp                       # last one as x^n
            Srow[j] = SL[0]
        pass

    for k in range(num-1,num):                  # can generate v large matrices, 2^8-1, 2^9-1
        n = 2**k-1
        Srow = np.zeros(n,dtype=int)            # holds one row of S matrix
        if k in [2,3,4,6,7,15]:	S_lineA(k,1)
        if k in [5,11]:	S_lineA(k,2)
        if k == 8:      S_lineB(k,1,5,6)
        if k == 9:      S_lineA(k,4)
        if k == 10:     S_lineA(k,3)
        if k == 12:     S_lineB(k,3,4,7)
        if k == 13:     S_lineB(k,1,3,4)
        if k == 14:     S_lineB(k,1,11,12)
        pass

        S = np.zeros((n,n),dtype = int)
        for i in range(n):
            for j in range(n):
                S[i,j] = Srow[n-1-j]
            Srow = np.roll(Srow, 1)                   # rotate by 1 element at a time
            pass

    return S
#------------
# 2^k+1  is size of matrix side
S = shift_hadamard(4)
#print(S)

In [122]:
# Matrix doubling method
# S matrix size (2^k) -1 , k = 2, 3, 4 etc, replace each element by previous matrix
def doubling_hadamard(max_size):

    #print('Matrix doubling method size (2^k)-1, k=2,3.. :  k=',max_size , 'S matrix is not circulant')
    max_size = int(max_size)
    h0 = np.ones( (2,2),dtype = int )     # define initial Hadamard matrix
    h0[1,0] = -1
    #h0[1,1]=-1    # this way round leading row & col are 1's
    #print('initial form\n',h0,h0 @ h0.T)
    n = 2  # must be 2
    Htemp = h0
    for i in range(1,max_size):
        #print(' {:s} {:d}'.format(' calculation continues, S size = ',2*2**i-1))
        Hnn = np.zeros((2*n,2*n),dtype=int)
        h0, h1 = Htemp.shape
        Hnn[0  : 0  + h0   , 0  : 0  + h1] =  Htemp
        Hnn[n  : n  + h0   , 0  : 0  + h1] = -Htemp
        Hnn[0  : 0  + h0   , n  : n  + h1] =  Htemp
        Hnn[n  : n  + h0   , n  : n  + h1] =  Htemp
        #print('Hnn\n',Hnn)
        n = 2*n
        s = Hnn.shape
        Htemp = np.zeros( s ,dtype = int)
        Htemp = Hnn[ 0:s[0],  0:s[0] ]

        sm = s[0] - 1
        Tmpmat = np.zeros((sm,sm),dtype=int)
        Smat   = np.zeros((sm,sm),dtype=int)
        Tmpmat= Htemp[1:sm+1, 0:sm]
        for ii in range(sm):
            for j in range(sm):
                if Tmpmat[ii,j] == 1:
                    Smat[ii,j]= 0
                else:
                    Smat[ii,j]= 1
                pass
    #print('-------------------------------')
    return Smat
#-------------

smat = doubling_hadamard(4)                   # 4 means  15 x 15 matrix
#print('final S matrix \n',smat.shape)
#print('\n'.join( [''.join(['{:2}'.format(item) for item in row] ) for row in smat] ) )

In [123]:

# type-2 reduced S matrix generation
def make_rows(m):                # type 2 matrix
    if m == 7:
        letters = ['A','D','F','G']
    if m == 11:
        letters = ['A', 'C', 'G', 'H', 'I', 'K']                        #  as in table in paper
    if m == 15:
        #letters = ['A', 'B', 'C', 'D', 'E', 'F','G','H','M','N']       # best on 3 and 4 groups
        letters = ['A', 'B', 'C', 'D', 'E', 'F', 'I', 'J', 'K', 'O']    # 4 with 4 + 6 with 6.
        #letters = ['A', 'B', 'C', 'D', 'E', 'F', 'H', 'J', 'K', 'N']   # doubling method 4 x 6 + 4*4
    if m == 19:
        #letters = ['A', 'D', 'G', 'I', 'K', 'M','N','O','P','S']
        letters = ['A', 'C', 'D', 'I', 'K', 'M', 'N', 'O', 'P', 'S']    # all fives
    if m == 23:
        letters = ['A','F','H','I','K','O','P','R','T','V','W']
    if m == 27:
        letters = ['A','B','C','D','E','F','G','H','J','M','O','T','X']
    if m == 31:
        letters = ['C','D','G','J','L','N','P','U','V','X','Y','a','e']
    rows    = np.zeros(len(letters),dtype=int)
    for i in range(len(rows) ):
        if ord(letters[i]) < 91:                                 # ascii A = 65,  Z = 90
            rows[i] = ord(letters[i]) - 65                       # is capital A = 1, Z = 26
        else:
            rows[i] = ord(letters[i]) - 97 + 27 -1               # ascii 97 is lower case a
    return rows
#---------------------------------
def make_rows_type1(m):                # type 1 matrix
    if m == 7:
        letters = ['A','D','F']
    if m == 11:
        letters = ['B', 'D', 'E', 'G']
    if m == 15:
        letters = ['B', 'C', 'D', 'E']
    if m == 19:
        letters = ['B', 'E', 'F', 'H', 'N']
    if m == 23:
        letters = ['A','H','K','N','Q']
    if m == 27:
        letters = ['A','B','E','N','V']
    if m == 31:
        letters = ['B','C','E','G','L','O']
    rows    = np.zeros(len(letters),dtype=int)
    for i in range(len(rows) ):
        if ord(letters[i]) < 91:                                 # ascii A = 65,  Z = 90
            rows[i] = ord(letters[i]) - 65                       # is capital A => 1, Z => 26
        else:
            rows[i] = ord(letters[i]) - 97 + 27 - 1               # ascii 97 is lower case a
    return rows
#---------------------------------

def make_reduced_matrix(m,rows,S):                       # Reduced S matrix, remove last column
    Sred = np.zeros((len(rows),m-1), dtype =int)
    for k,val in enumerate(rows):
        arry = S[val, 0:m - 1]
        Sred[k, 0:m - 1] = arry
        pass
    return Sred
#--------------------------

def do_row_sums(data,Sred):                   # Calculate down columns to get sum
    row, col = Sred.shape
    #print('row col', row,col)
    dotp = np.zeros(row, dtype=float)
    for i in range(row):                      # dot product. Make vector dotp
        dotp[i] = np.dot(Sred[i,:] , data[:])
    return dotp
#-----------------------------

def test_for_positives(data,Sred,pnum):

    dotpvec = do_row_sums(data,Sred)
    if sum(dotpvec) == 0:
        temp = np.zeros(len(dotpvec),dtype=int)# 0 positive
        indx = [-10 for i in range(10)]
        indx[0] = -2
        flag = 0
        print(temp,' Zero pos')
        pnum[0] = pnum[0] + 1
    else:                                               # samples present

        pnum, q,   indx, flag,total_column,total_numbs,total_ones = one_sample(Sred,   dotpvec,comb_indx1,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            pnum, q,indx,flag,total_column,total_numbs,total_ones = two_samples(Sred,  dotpvec,comb_indx2,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            pnum, q,indx,flag,total_column,total_numbs,total_ones = three_samples(Sred,dotpvec,comb_indx3,pnum,total_column,total_numbs,total_ones)
        if q == 0:
            print('more than 3,  vector =',dotpvec)
            indx = [-10 for i in range(10)]
            indx[0] = -1
            pnum[7] = pnum[7] + 1
            flag = 1
    return pnum ,indx,flag                                            # number of positives in this array
#--------------------------------------
def allcombs(alist, num):
    return list(combinations(alist,num))
#---------------------------------------


In [ ]:
# --- Legendre construction method for n = 27 ---

def legendre(p):
    """Legendre symbols, see Wikipedia."""
    leg = np.zeros(p, dtype=int)
    for i in range(p):
        temp = pow(i, (p-1)//2, p)   # integer exponentiation mod p
        if temp > 1:
            temp = -1
        leg[i] = temp
    return leg[1:]   # drop the 0-th element


def Hadamard28(L, p):
    """Construct a 28×28 Hadamard matrix, then reduce to 27×27 S-matrix."""
    Had28 = np.zeros((2*(p+1), 2*(p+1)), dtype=int)
    B     = np.zeros((p+1, p+1), dtype=int)
    Q     = np.zeros((p, p), dtype=int)

    # Build Q
    Q[0, 1:] = L[:]
    for i in range(1, p):
        Q[i, :] = np.roll(Q[0, :], i)
    Q[0, 0] = 0

    # Insert Q into B
    B[0, :] = 1
    B[:, 0] = 1
    for i in range(1, p+1):
        for j in range(1, p+1):
            B[i, j] = Q[i-1, j-1]
    B[0, 0] = 0

    # Fill Had28
    for i in range(0, p+1):
        for j in range(0, p+1):
            if B[i, j] == 1:
                Had28[i+i-1, j+j-1] = 1
                Had28[i+i-1, j+j]   = 1
                Had28[i+i,   j+j-1] = 1
                Had28[i+i,   j+j]   = 0
            elif B[i, j] == -1:
                Had28[i+(i-1), j+(j-1)] = 0
                Had28[i+(i-1), j+j]     = 0
                Had28[i+i,     j+(j-1)] = 0
                Had28[i+i,     j+j]     = 1
            elif B[i, j] == 0:
                Had28[i+(i-1), j+(j-1)] = 1
                Had28[i+(i-1), j+j]     = 0
                Had28[i+i,     j+(j-1)] = 0
                Had28[i+i,     j+j]     = 0
    return Had28


def generate_legendre_S():
    """Explicitly generate the S-matrix for n=27 using Legendre method."""
    p = 13
    L = legendre(p)
    Had28 = Hadamard28(L, p)
    S27 = Had28[:-1, :-1]   # drop last row & col → 27×27
    print(f"Generated {S27.shape[0]}×{S27.shape[1]} S-matrix using Legendre construction (n=27).")
    return S27


In [124]:
def _is_power_of_two(x: int) -> bool:
    return x > 0 and (x & (x - 1)) == 0


def generate_S(n: int) -> np.ndarray:
    """Generate an S-matrix of (about) size n using the most appropriate method.
    Priority:
      1) Legendre construction (special case n=27)
      2) Quadratic Residue (if valid_seq_length(n))
      3) Shift Register (if n = 2^k - 1)
      4) Doubling fallback (nearest 2^k - 1)
    """

    # --- Special case: n=27 (Legendre construction) ---
    if n == 27:
        p = 13
        L = legendre(p)
        Had28 = Hadamard28(L, p)
        S27 = Had28[:-1, :-1]   # drop last row & col → 27×27
        print("Generated 27×27 S-matrix using Legendre construction.")
        return S27

    # --- Quadratic Residue ---
    S = None
    try:
        if valid_seq_length(n):
            S = quadratic_hadamard(n)
            method = "Quadratic Residue"
    except Exception:
        S = None

    # --- Shift Register ---
    if S is None and _is_power_of_two(n + 1):
        k = int(np.log2(n + 1))
        S = shift_hadamard(k)
        method = f"Shift Register (k={k})"

    # --- Doubling fallback ---
    if S is None:
        k = int(np.rint(np.log2(n + 1)))
        k = max(k, 2)
        S = doubling_hadamard(k)
        method = f"Doubling (k={k})"

    print(f"Generated {S.shape[0]}×{S.shape[1]} S-matrix using {method}.")
    return S


In [125]:
#Generate type-2 reduced matrix for input n value
def reduced_via_make_rows(S: np.ndarray, n: int):
    """
    Type-2 reduction using your make_rows(n):
    - Select rows from S using make_rows(n) (auto-fix 1-based indices).
    - Drop the **last column** so the reduced matrix has (n-1) columns.
    """
    if 'make_rows' not in globals():
        raise RuntimeError('make_rows(n) is not defined. Please ensure the original cell is run.')

    rows = list(make_rows(n))

    # If make_rows returned 1-based indices, convert to 0-based
    if len(rows) and max(rows) >= S.shape[0]:
        rows = [r - 1 for r in rows]

    rows = [int(r) for r in rows]

    # Drop the last column (Type-2 reduction)
    R = S[np.ix_(rows, np.arange(S.shape[1] - 1))]
    return R, rows


In [126]:
#Generate the combination of samples that should be used for each pool
def pools_from_rows(rows0: list[int]) -> list[str]:
    return [chr(65 + r) for r in rows0]

def pool_assignments(R: np.ndarray):
    assigns = []
    for r in range(R.shape[0]):
        assigns.append([c+1 for c, v in enumerate(R[r, :]) if v == 1])
    return assigns

def column_matches(R: np.ndarray, v: np.ndarray) -> list[int]:
    return [j+1 for j in range(R.shape[1]) if np.array_equal(R[:, j], v)]

def sum_of_columns_matches(R: np.ndarray, v: np.ndarray, max_solutions: int = 20):
    R = np.array(R, dtype=int); v = np.array(v, dtype=int)
    cols = [j for j in range(R.shape[1]) if np.all(R[:, j] <= v)]
    cols.sort(key=lambda j: int(R[:, j].sum()), reverse=True)
    sols = []
    def dfs(start, residual, chosen):
        if len(sols) >= max_solutions:
            return
        if np.all(residual == 0):
            sols.append([c+1 for c in chosen]); return
        if np.any(residual < 0):
            return
        for idx in range(start, len(cols)):
            j = cols[idx]; col = R[:, j]
            if np.all(col <= residual):
                dfs(idx+1, residual - col, chosen + [j])
    dfs(0, v, [])
    return sols


## Matrix Generator and Analysis Vector Checking Tool

In [127]:
n = 7  # <-- change here or run the widget cell below
S = generate_S(n)
print('Full S-matrix (shape:', S.shape, ')\n', S)
R, rows = reduced_via_make_rows(S, n)
labels = pools_from_rows(rows)
print('\nReduced matrix via make_rows (rows:', labels, ') shape:', R.shape, '\n', R)
assigns = pool_assignments(R)
print('\nPool assignments (samples to add):')
for lab, samp in zip(labels, assigns):
    print(f'  Pool {lab}: samples {samp}')
if R.shape[1] > 0:
    example = R[:, 0]
    print('\nExample analysis vector (pool pattern of Sample 1):', list(map(int, example)))


Generated 7×7 S-matrix using Quadratic Residue.
Full S-matrix (shape: (7, 7) )
 [[0 0 1 0 1 1 1]
 [0 1 0 1 1 1 0]
 [1 0 1 1 1 0 0]
 [0 1 1 1 0 0 1]
 [1 1 1 0 0 1 0]
 [1 1 0 0 1 0 1]
 [1 0 0 1 0 1 1]]

Reduced matrix via make_rows (rows: ['A', 'D', 'F', 'G'] ) shape: (4, 6) 
 [[0 0 1 0 1 1]
 [0 1 1 1 0 0]
 [1 1 0 0 1 0]
 [1 0 0 1 0 1]]

Pool assignments (samples to add):
  Pool A: samples [3, 5, 6]
  Pool D: samples [2, 3, 4]
  Pool F: samples [1, 2, 5]
  Pool G: samples [1, 4, 6]

Example analysis vector (pool pattern of Sample 1): [0, 0, 1, 1]


In [128]:
def _check(_):
    with out:
        if state['R'] is None:
            print('Build first'); return

        R = state['R']

        # Accept comma- or space-separated input (or both mixed)
        tokens = [t for t in re.split(r'[,\s]+', rv_widget.value.strip()) if t != '']
        try:
            v = np.array([int(t) for t in tokens], dtype=int)
        except:
            print('Use comma- or space-separated integers'); return

        if v.size != R.shape[0]:
            print(f'Length {v.size} must equal number of pools {R.shape[0]}'); return

        print('Results:', _fmt(v))

        # 1) Single-sample matches
        singles = column_matches(R, v)
        if singles:
            print('Exact single-sample match(es):', singles)

        # 2) Double matches
        doubles = []
        for i in range(R.shape[1]):
            for j in range(i+1, R.shape[1]):
                if np.array_equal(R[:, i] + R[:, j], v):
                    doubles.append((i+1, j+1))
        if doubles:
            print('Exact double-sample match(es):')
            for d in doubles:
                print('  •', d)

        # 3) General multi-sample matches
        sols = sum_of_columns_matches(R, v)
        if sols:
            print('Set(s) of positives whose sum matches:')
            for s in sols:
                print('  •', s)

        if not singles and not doubles and not sols:
            print('No exact match')


In [130]:
# --- Interactive widget version with support for 0/1/2/3/... ---
import re

n_widget = wgt.IntText(value=7, description='n')
build_btn = wgt.Button(description='Build')
rv_widget = wgt.Text(
    value='',
    description='Results v',
    placeholder='e.g. 0,1,2,0 or 0 1 2 0'
)
check_btn = wgt.Button(description='Check')
out = wgt.Output()

state = {'S': None, 'R': None, 'labels': None}

def _fmt(v):
    return '[' + ', '.join(map(str, map(int, v))) + ']'

def _build(_):
    with out:
        out.clear_output()
        n = int(n_widget.value)
        S = generate_S(n)
        print('Full S-matrix\n', S)

        R, rows = reduced_via_make_rows(S, n)
        labels = pools_from_rows(rows)
        print('\nReduced (rows', labels, ')\n', R)

        print('\nPool assignments:')
        for lab, samp in zip(labels, pool_assignments(R)):
            print(f'  Pool {lab}: samples {samp}')

        # Save state
        state.update(S=S, R=R, labels=labels)

def _check(_):
    with out:
        if state['R'] is None:
            print('Build first'); return

        R = state['R']

        # Accept comma- or space-separated input (or both mixed)
        tokens = [t for t in re.split(r'[,\s]+', rv_widget.value.strip()) if t != '']
        try:
            v = np.array([int(t) for t in tokens], dtype=int)
        except:
            print('Use comma- or space-separated integers'); return

        if v.size != R.shape[0]:
            print(f'Length {v.size} must equal number of pools {R.shape[0]}'); return

        print('Results:', _fmt(v))

        # 1) Single-sample matches
        singles = column_matches(R, v)
        if singles:
            print('Exact single-sample match(es):', singles)

        # 2) Double matches
        doubles = []
        for i in range(R.shape[1]):
            for j in range(i+1, R.shape[1]):
                if np.array_equal(R[:, i] + R[:, j], v):
                    doubles.append((i+1, j+1))
        if doubles:
            print('Exact double-sample match(es):')
            for d in doubles:
                print('  •', d)

        # 3) General multi-sample matches
        sols = sum_of_columns_matches(R, v)
        if sols:
            print('Positive Samples:')
            for s in sols:
                print('  •', s)

        if not singles and not doubles and not sols:
            print('No exact match')

build_btn.on_click(_build)
check_btn.on_click(_check)

display(n_widget, build_btn, rv_widget, check_btn, out)


IntText(value=7, description='n')

Button(description='Build', style=ButtonStyle())

Text(value='', description='Results v', placeholder='e.g. 0,1,2,0 or 0 1 2 0')

Button(description='Check', style=ButtonStyle())

Output()